In [1]:
import numpy as np

In [2]:
def import_txt(file_str):
    '''Read files as hex arrays. Returns an array of tracks with 28 hex strings per track.
    First 12 hex strings are track fit constants, and last 16 hex strings are hit coordinates.'''
    results = []
    with open(file_str) as inputfile:
        for line in inputfile:
            results.append(line.strip()[-4:].split(','))
    track_results = results[14:len(results)-16]
    n_track = int(np.floor(len(track_results)/28))
    track_v = np.array(track_results).reshape(n_track,-1)
    return track_v

In [3]:
track_v = import_txt('ftksim_3tracks.txt')
hw_v = import_txt('ftksim_3tracks_hwout.txt')

In [4]:
def matching_track_clusters(track_v):
    '''Given an array of tracks, return a list of matching (sharing 6 or more hits) track indices.
    e.g. Given 5 tracks, where tracks 0 and 1 match, and tracks 3 and 4 match, return [[0,1], [2], [3,4]].
    Note that if tracks 0 and 1 match, and tracks 1 and 2 match, we consider tracks 0 and 2 to match.'''
    def n_matching_hits(track_i, track_j):
        '''Number of matching hits (IBL+SCT) between two tracks.'''
        matches = (track_i[12:] == track_j[12:])
        n_IBL_matches = (matches[0]&matches[1])+(matches[2]&matches[3])+(matches[4]&matches[5])+(matches[6]&matches[7])
        n_SCT_matches = sum(matches[8:16])
        return n_IBL_matches + n_SCT_matches
    track_clusters = []
    for i in range(len(track_v)):
        current_cluster = []
        for j in range(i, len(track_v)):
            if n_matching_hits(track_v[i], track_v[j])>6: current_cluster.append(j)
        is_new_cluster = True
        for n, cluster in enumerate(track_clusters):
            combined_cluster = set(cluster+current_cluster)
            if len(combined_cluster) < len(cluster) + len(current_cluster):
                track_clusters[n] = list(combined_cluster)
                is_new_cluster = False
                break
        if is_new_cluster: track_clusters.append(current_cluster)
    track_clusters = [np.array(i) for i in track_clusters]
    return track_clusters

def best_track(track_cluster, track_v):
    '''For each track cluster, return the index of the track with the best fit parameters (lowest chi2 out of
    tracks with most layer map hits).'''
    def hex2bin(hex_num):
        '''Replaces each character of a hex string (ignoring the leading 0) with four bits.
        e.g. hex2bin("0af1") = "101011110001".'''
        def hex2bin_map(x):
            '''Interprets a character as hex and converts to a padded 4-digit binary string.'''
            return bin(int("0x"+x,16))[2:].zfill(4)
        bin_num = ""
        for i in range(1,4):
            bin_num = bin_num+hex2bin_map(hex_num[i])
        return bin_num
    def layer_map_sum(layer_map):
        '''Sums up the number of 1\'s in a string.
        e.g. "111101110111" returns 11.'''
        return layer_map.count("1")
    layer_map_sum_v = [layer_map_sum(hex2bin(track_v[cluster,3])) for cluster in track_cluster]
    chisq_v = [track_v[cluster,6] for cluster in track_cluster]
    indices_with_max_layer_map_hits = np.where([i==max(layer_map_sum_v) for i in layer_map_sum_v])[0]
    best_index = chisq_v.index(min(np.array(chisq_v)[indices_with_max_layer_map_hits]))
    return track_cluster[best_index]

def HitWarrior(track_v):
    '''Given a set of tracks, we cluster them into matching tracks, and find the best track from each cluster.'''
    track_clusters = matching_track_clusters(track_v)
    best_tracks = [best_track(cluster, track_v) for cluster in track_clusters]
    return track_clusters, best_tracks

In [5]:
track_clusters, HW_tracks = HitWarrior(track_v)
print(list(track_clusters))
print(list(HW_tracks))

[array([ 0,  6,  7,  9, 12, 16]), array([1]), array([ 2,  3,  4, 17, 18, 20, 21, 22]), array([5]), array([ 8, 10, 11, 13, 14]), array([15, 19, 23, 24, 25, 26, 27])]
[16, 1, 18, 5, 8, 25]
